In [4]:
# Data Collection

!pip install kaggle --upgrade
!pip3 install Pillow

import kaggle
import os
import json
import random
from datetime import datetime
from PIL import Image, ImageStat
from PIL.ExifTags import TAGS
from scipy.spatial import KDTree
from webcolors import (
    CSS3_HEX_TO_NAMES,
    hex_to_rgb,
)
def get_predominant_colors(image_file, numcolors=3, resize=150, output = []):
    # Resize image to speed up processing
    img = Image.open(image_file)
    img = img.copy()
    img.thumbnail((resize, resize))
 
    # Reduce to palette
    paletted = img.convert('P', palette=Image.ADAPTIVE, colors=numcolors)
 
    # Find dominant colors
    palette = paletted.getpalette()
    color_counts = sorted(paletted.getcolors(), reverse=True)
    colors = list()
    for i in range(numcolors):
        palette_index = color_counts[i][1]
        dominant_color = palette[palette_index*3:palette_index*3+3]
        colors.append(tuple(dominant_color))
    return colors

def convert_rgb_to_names(rgb_tuple):
    
    # a dictionary of all the hex and their respective names in css3
    css3_db = CSS3_HEX_TO_NAMES
    names = []
    rgb_values = []    
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(hex_to_rgb(color_hex))
    
    kdt_db = KDTree(rgb_values)    
    distance, index = kdt_db.query(rgb_tuple)
    return int(index)

In [5]:
## You have to collect and download a set of images. You have the following tasks to program, automating the process as much as possible:
### 1. Create a folder called images.

!mkdir -p ./images
!mkdir -p ./metadata

### 2. Download open-licensed images to the folder images (minimum 100 images).

kaggle.api.authenticate()
kaggle.api.dataset_download_files('nielspace/pexels-mountain-images', path='./images', unzip=True)    
    
### 3. Save metadata of every image like image size, image format (.jpeg, .png, etc.), image orientation (landscape, portrait, square, etc.), creation date, camera model, etc. in one or more JSON files. You can make use of the Exif information present in the image files.

directory = './images/Mountain'
metadata = {}
orientation = ""
images_data =[]
for filename in os.listdir(directory) :
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        image_data = {}
        path, file_extension = os.path.splitext(f)
        time_creation = os.path.getmtime(f)
        im = Image.open(f)
        size = im.size
        colors = {}
        predominant_colors = get_predominant_colors(directory + "/" + filename)
        colors["predominant_colors"] = []
        for predominant_color in predominant_colors:
            colors["predominant_colors"].append(convert_rgb_to_names(predominant_color))
        for idx,val in enumerate(["red","green","blue"]):
            colors[val] = Image.Image.getextrema(im)[idx]
            colors[val] = colors[val] + (ImageStat.Stat(im).median[idx],)
        print(colors)
        if size[0]>size[1] :
            orientation = "landscape"
        elif size[0] == size[1] :
            orientation = "square"
        else :
            orientation = "portrait"
        metadata[filename] = {
            "file_extension": file_extension,
            "creation_date": datetime.fromtimestamp(time_creation/1000.0).strftime("%m/%d/%Y, %H:%M:%S"),
            "size": size,
            "orientation": orientation,
            "colors": colors
            }


with open("./metadata/metadata.json", "w") as outfile:
    json.dump(metadata, outfile, indent=4)

        

{'predominant_colors': [27, 25, 12], 'red': (0, 255, 169), 'green': (0, 255, 141), 'blue': (0, 255, 72)}
{'predominant_colors': [34, 72, 120], 'red': (0, 255, 115), 'green': (0, 255, 122), 'blue': (0, 255, 129)}
{'predominant_colors': [23, 46, 58], 'red': (15, 255, 216), 'green': (12, 255, 215), 'blue': (17, 255, 219)}
{'predominant_colors': [126, 34, 79], 'red': (0, 255, 24), 'green': (0, 255, 91), 'blue': (0, 255, 171)}
{'predominant_colors': [27, 58, 66], 'red': (11, 255, 170), 'green': (6, 253, 157), 'blue': (0, 252, 150)}
{'predominant_colors': [34, 66, 72], 'red': (8, 255, 84), 'green': (15, 255, 83), 'blue': (0, 255, 79)}
{'predominant_colors': [34, 45, 72], 'red': (0, 255, 90), 'green': (0, 255, 90), 'blue': (0, 255, 118)}
{'predominant_colors': [7, 72, 34], 'red': (0, 255, 2), 'green': (0, 255, 27), 'blue': (0, 255, 39)}
{'predominant_colors': [27, 120, 112], 'red': (12, 255, 150), 'green': (9, 235, 137), 'blue': (0, 239, 129)}
{'predominant_colors': [34, 73, 45], 'red': (0, 2

{'predominant_colors': [126, 33, 66], 'red': (0, 255, 63), 'green': (0, 255, 100), 'blue': (4, 255, 154)}
{'predominant_colors': [45, 66, 77], 'red': (0, 255, 217), 'green': (9, 255, 223), 'blue': (13, 255, 225)}
{'predominant_colors': [27, 39, 23], 'red': (0, 255, 101), 'green': (0, 252, 111), 'blue': (0, 246, 86)}
{'predominant_colors': [23, 39, 120], 'red': (0, 255, 161), 'green': (2, 255, 164), 'blue': (0, 255, 166)}
{'predominant_colors': [34, 109, 23], 'red': (0, 255, 130), 'green': (0, 255, 113), 'blue': (0, 255, 98)}
{'predominant_colors': [34, 66, 45], 'red': (0, 255, 195), 'green': (1, 255, 202), 'blue': (2, 255, 203)}
{'predominant_colors': [126, 45, 73], 'red': (0, 255, 148), 'green': (0, 255, 188), 'blue': (0, 255, 201)}
{'predominant_colors': [34, 27, 49], 'red': (0, 255, 67), 'green': (0, 255, 91), 'blue': (0, 254, 38)}
{'predominant_colors': [34, 123, 23], 'red': (0, 255, 70), 'green': (0, 255, 86), 'blue': (0, 255, 98)}
{'predominant_colors': [34, 77, 112], 'red': (22,

In [8]:
# Labeling and Annotation

directory = './images/Mountain'
data_path = "./metadata/metadata.json"


if os.path.isfile(data_path):
    with open(data_path) as target:
        json_data = json.load(target)
        
for filename in os.listdir(directory) :
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        tags = {}
        tags["like"] = random.randint(0,100)
        tags["hashtag"] = "moutain"
        rand = random.randint(1, 3)
        if rand == 1 :
            tags["test"] = "yes"
        else :
            tags["test"] = "no"
        json_data[filename]["tags"] = tags
        
with open("./metadata/metadata.json", "w") as outfile:
    json.dump(json_data, outfile, indent=4)

In [9]:
from sklearn import tree
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Data Analyses

numb_users = 1
directory = './images/Mountain'
data_path = "./metadata/metadata.json"
rand = 0

# Metadata for each file
if os.path.isfile(data_path):
    with open(data_path) as target:
        json_data = json.load(target)

# Creating data for each user
json_data_users = {}        
for i in range (0, numb_users):
    images_per_users = []
    tags_per_users = []
    rand = random.randint(1, 4)
    if rand == 1:
        tags_per_users.append(["like", "colors"])
    elif rand == 2:
        tags_per_users.append("colors")
    elif rand == 3:
        tags_per_users.append(["hashtag", "colors"])
    else:
        tags_per_users.append(["like, hashtag", "colors"])
    for filename in os.listdir(directory) :
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            rand = random.randint(1, 3)
            if rand == 1 :
                images_per_users.append(filename)
                print(filename)
    json_data_users[i] = {
        "images": images_per_users,
        "tags": tags_per_users
    }


pexels-eberhard-grossgasteiger-2310641.jpg
pexels-pixabay-355482.jpg
pexels-eberhard-grossgasteiger-1366909.jpg
pexels-markus-spiske-143577.jpg
pexels-jarod-lovekamp-3791466.jpg
pexels-robby-mccullough-1867601.jpg
pexels-alessio-cesario-1906794.jpg
pexels-eberhard-grossgasteiger-572897.jpg
pexels-luis-ruiz-991422.jpg
pexels-tyler-lastovich-633198.jpg
pexels-kasuma-427676.jpg
pexels-tim-gouw-291732.jpg
pexels-tyler-lastovich-937782.jpg
pexels-tom-verdoot-1446713.jpg
pexels-quang-nguyen-vinh-2649403.jpg
pexels-eberhard-grossgasteiger-1699021.jpg
pexels-eberhard-grossgasteiger-1146642.jpg
pexels-eberhard-grossgasteiger-1428277.jpg
pexels-pixabay-417265.jpg
pexels-vittorio-staffolani-655674.jpg
pexels-eberhard-grossgasteiger-640858.jpg
pexels-pixabay-36478.jpg
pexels-archie-binamira-913215.jpg
pexels-quang-nguyen-vinh-2166695.jpg
pexels-pixabay-158272.jpg
pexels-eberhard-grossgasteiger-2088210.jpg
pexels-zhaocan-li-1755243.jpg
pexels-pixabay-355747.jpg
pexels-krivec-ales-547115.jpg
pexels-

In [10]:
# Creating dataframes to predict what the user might like
data = []
result = []
json_data_learning = dict(list(json_data.items())[len(json_data)//3:])
for i in json_data_learning:
    if i in json_data_users[0]["images"]:
        result.append('Favorite')
    else:
        result.append('NotFavorite')
    data.append([json_data_learning[i]["colors"]["predominant_colors"][0],
                 json_data_learning[i]["colors"]["predominant_colors"][1],
                 json_data_learning[i]["colors"]["predominant_colors"][2],
                 json_data_learning[i]["colors"]["red"][1],
                 json_data_learning[i]["colors"]["red"][2],
                 json_data_learning[i]["colors"]["green"][1],
                 json_data_learning[i]["colors"]["green"][2],
                 json_data_learning[i]["colors"]["blue"][1],
                 json_data_learning[i]["colors"]["blue"][2],
                 json_data_learning[i]["orientation"], 
                 json_data_learning[i]["size"][0],
                 json_data_learning[i]["size"][1]
                ])

dataframe = pd.DataFrame(data, columns=['first_color', 'second_color', 'third_color','max_red', 'median_red','max_green', 'median_green','max_blue', 'median_blue','orientation', 'width', 'height'])
resultframe = pd.DataFrame(result, columns=['favorite'])
dataframe

,first_color,second_color,third_color,max_red,median_red,max_green,median_green,max_blue,median_blue,orientation,width,height
0,62,73,72,255,166,254,193,255,220,landscape,6000,4000
1,124,134,23,255,252,255,252,255,252,portrait,4160,5200
2,34,120,112,255,125,255,116,255,109,portrait,3648,5472
3,72,33,23,255,103,251,123,255,150,landscape,7782,5191
4,34,66,45,242,191,243,192,245,194,portrait,3648,5472
...,...,...,...,...,...,...,...,...,...,...,...,...
63,7,23,27,255,52,255,29,255,8,landscape,4935,3293
64,23,120,45,255,205,255,183,254,188,landscape,3648,2432
65,34,49,127,255,102,255,122,255,115,landscape,7952,5304
66,34,66,39,255,55,250,69,255,96,portrait,2048,2560


In [ ]:
import os
import math 
import matplotlib.pyplot as plot
isLiked = resultframe["favorite"]=="Favorite"
dataframeLiked = dataframe.filter(items = resultframe[isLiked]["favorite"].index, axis = 0)
grouped = dataframeLiked.groupby(['orientation']).count()
dataframeLiked
grouped = grouped.rename(columns={'width':'count'})["count"].reset_index()
grouped.plot(x=0, kind='bar', title="Orientations liked")

In [ ]:
#generating numerical labels
le1 = LabelEncoder()
dataframe['orientation'] = le1.fit_transform(dataframe['orientation'])

le2 = LabelEncoder()
resultframe['favorite'] = le2.fit_transform(resultframe['favorite'])

dtc = tree.DecisionTreeClassifier()
dtc = dtc.fit(dataframe, resultframe)

prediction = dtc.predict([
        [1,
         2,
         3,
         le1.transform(['landscape'])[0],
         12,
         23
        ]])
print(le2.inverse_transform(prediction))
print(dtc.feature_importances_)